In [1]:
import hdf5plugin
import h5py
import numpy as np
import pandas as pd

In [2]:
filename = "train_cite_inputs.h5"

with h5py.File(filename,'r') as hdf:
    ls = list(hdf.keys())
#     print("list of datasets in this file : \n",ls)
#     print("list of keys in dataset ",hdf.get('train_cite_inputs').keys())
    
#     print(hdf['train_cite_inputs']['axis0'])
    axis0 = hdf.get('train_cite_inputs').get('axis0')
    axis0_arr = np.array(axis0)
    axis0_arr = np.char.decode(axis0_arr) 
#     print(axis0_arr) 
    
#     print(hdf['train_cite_inputs']['axis1'])
    axis1 = hdf.get('train_cite_inputs').get('axis1')
    axis1_arr = np.array(axis1)
    axis1_arr = np.char.decode(axis1_arr)
#     print(axis1_arr)
    
#     print(hdf['train_cite_inputs']['block0_items'])
    block0_items = hdf.get('train_cite_inputs').get('block0_items')
    block0_items_arr = np.array(block0_items)
#     print(block0_items_arr)
    
    
#     print(hdf['train_cite_inputs']['block0_values'])
    block0_values = hdf.get('train_cite_inputs').get('block0_values')
    block0_values_arr = np.array(block0_values)
#     print(block0_values_arr)

In [3]:
df_train_input = pd.DataFrame(columns=axis0_arr, data=block0_values_arr,index=axis1_arr)
# print(df_train_input.head(101))
# print(list())

In [4]:
filename = "train_cite_targets.h5"

with h5py.File(filename,'r') as hdf:
    ls =list(hdf.keys())
#     print("list of datasets in this file : \n",ls)
#     print("list of keys in dataset ",hdf.get('train_cite_targets').keys())
#     print(hdf['train_cite_targets']['axis0'])
    
    axis0 = hdf.get('train_cite_targets').get('axis0')
    axis0_arr = np.array(axis0)
    axis0_arr = np.char.decode(axis0_arr) 
#     print(axis0_arr) 
    
#     print(hdf['train_cite_targets']['axis1'])
    axis1 = hdf.get('train_cite_targets').get('axis1')
    axis1_arr = np.array(axis1)
    axis1_arr = np.char.decode(axis1_arr)
#     print(axis1_arr)
    
#     print(hdf['train_cite_targets']['block0_items'])
    block0_items = hdf.get('train_cite_targets').get('block0_items')
    block0_items_arr = np.array(block0_items)
#     print(block0_items_arr)
    
    
#     print(hdf['train_cite_targets']['block0_values'])
    block0_values = hdf.get('train_cite_targets').get('block0_values')
    block0_values_arr = np.array(block0_values)
#     print(block0_values_arr)

In [5]:
df_train_target = pd.DataFrame(columns=axis0_arr, data=block0_values_arr,index=axis1_arr)
target_columns = list(df_train_target.columns.values)

In [6]:
print(len(df_train_input), len(df_train_target))

70988 70988


In [7]:
sample_size = 70988
df_train_input = df_train_input.iloc[:sample_size]
df_train_target = df_train_target.iloc[:sample_size]

#numero uno leaderboard preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler,normalize

# df_norm = StandardScaler().fit_transform(df_train_input.values)
df_norm = pd.DataFrame(normalize(df_train_input,norm='l2',axis=1),index=df_train_input.index, columns=df_train_input.columns)
df_median = df_norm.sub(df_norm.median(axis=0), axis=1)

tsvd = TruncatedSVD(n_components=128)
df_tsvd = pd.DataFrame(tsvd.fit_transform(df_median), index=df_median.index)
print("finished preprocessing")
df_train_input = df_tsvd
#order by index (cell_id)
df_train_input.sort_index(axis=0,inplace=True)
df_train_target.sort_index(axis=0,inplace=True)





df_merged = pd.merge(df_train_input,df_train_target,left_index=True, right_index=True)
df_merged.head(100)


from catboost import CatBoostRegressor

target_columns = df_train_target.columns.values
# print(target_columns)
# Separate the input and target columns
X = df_merged.drop(target_columns, axis=1)
y = df_merged[target_columns]




finished preprocessing


In [9]:
print("starting MLP Regressor...")
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats  import pearsonr

# Initialize the MLPRegressor with best params
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlp_model = MLPRegressor(hidden_layer_sizes=(100,100),activation='tanh', solver='lbfgs', max_iter=1000,
                        alpha=0.2)

# {'activation': 'tanh', 'alpha': 0.2, 'hidden_layer_sizes': (100, 100), 'max_iter': 500, 'solver': 'lbfgs'}
# best r2 score :  0.1276774082288487

# Train the models on the dataset
mlp_model.fit(X_train, y_train)

# Predict the target values using the trained models
mlp_y_preds = mlp_model.predict(X_test)

#get model accuracy
# accuracy = accuracy_score(y_test, mlp_y_preds)
mae = mean_absolute_error(y_test, mlp_y_preds)
mse = mean_squared_error(y_test, mlp_y_preds)
# print("accuracy: ", accuracy)
print("mae: ", mae)
print("mse: ", mse)

#accuracy for multiple columns

threshold = 0.1
corr_loss = []
# print(y_test.shape)
# print(type(y_test))
# print(y_test.iloc[:10, 1])
# print(type(mlp_y_preds))
# print(mlp_y_preds.shape)
# print(mlp_y_preds[:10,1])
for i in range(y_test.shape[1]):  
    correlation_loss_i = 1 - pearsonr(y_test.iloc[:, i], mlp_y_preds[:,i])[0]
    corr_loss.append(correlation_loss_i)
print("Mean Correlation Loss: ", np.mean(corr_loss))
# print("successfull preds: ", np.mean(percentage_list))

starting MLP Regressor...


c:\python3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


mae:  1.026996666152159
mse:  2.446220031372783
Mean Correlation Loss:  0.5743312840233705


In [10]:
print("starting MLP Regressor...")
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats  import pearsonr

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#param optimization
param_grid = {
    'hidden_layer_sizes': [(50,),(100,),(50,50), (100,100), (64, 128)],
    'activation':['relu','logistic','tanh'],
    'solver':['adam','lbfgs','sgd'],
    'max_iter':[50,100,200,300,500],
    'alpha':[0.1, 0.2, 0.25],
}
# Initialize the MLPRegressor
# mlp_model = MLPRegressor(hidden_layer_sizes=(64, 128),activation='relu', solver='adam', learning_rate='adaptive', max_iter=500,
#                         alpha=0.01, early_stopping=True,validation_fraction=0.2)
mlp_model  = MLPRegressor()
grid_search = GridSearchCV(mlp_model, param_grid, cv = 5,n_jobs=-1)
# Train the models on the dataset
grid_search.fit(X_train, y_train)

print("best params : " , grid_search.best_params_)
print("best r2 score : ", grid_search.best_score_)

starting MLP Regressor...
best params :  {'activation': 'tanh', 'alpha': 0.2, 'hidden_layer_sizes': (100, 100), 'max_iter': 500, 'solver': 'lbfgs'}
best r2 score :  0.1276774082288487


c:\python3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [9]:
#calculate and divide by nonzero median
row_median = lambda row: np.median(row[row != 0])
nonzero_medians = df_train_input.apply(row_median,axis=1) 
df_div= df_train_input.div(nonzero_medians,axis=0) 
df_div.head(10)

,ENSG00000121410_A1BG,ENSG00000268895_A1BG-AS1,ENSG00000175899_A2M,ENSG00000245105_A2M-AS1,ENSG00000166535_A2ML1,ENSG00000128274_A4GALT,ENSG00000094914_AAAS,ENSG00000081760_AACS,ENSG00000109576_AADAT,ENSG00000103591_AAGAB,...,ENSG00000153975_ZUP1,ENSG00000086827_ZW10,ENSG00000174442_ZWILCH,ENSG00000122952_ZWINT,ENSG00000198205_ZXDA,ENSG00000198455_ZXDB,ENSG00000070476_ZXDC,ENSG00000162378_ZYG11B,ENSG00000159840_ZYX,ENSG00000074755_ZZEF1
000141b198e6,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,...,0.0,0.0,0.000000,1.271958,0.0,0.000000,0.00000,0.00000,0.000000,0.0
0003fcc59086,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.842476,...,0.0,1.0,0.842476,1.252469,0.0,0.000000,0.00000,0.00000,0.000000,0.0
000441bba05a,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.0
0004897a2b2b,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,1.000000,0.0
000517abfb58,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,...,0.0,0.0,1.000000,0.000000,0.0,0.000000,0.00000,1.00000,1.000000,0.0
00056fb385f2,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,1.00000,0.00000,1.000000,0.0
0005b45cd15b,0.0,0.0,0.0,0.0,0.0,0.0,1.104063,1.000000,0.0,0.000000,...,0.0,0.0,1.000000,1.358000,0.0,0.000000,0.82448,0.82448,1.000000,0.0
00097bbec9a2,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.838487,...,0.0,1.0,0.000000,0.838487,0.0,0.838487,0.00000,0.00000,1.000000,0.0
0009a974f266,0.0,0.0,0.0,0.0,0.0,0.0,0.838241,0.838241,0.0,0.000000,...,1.0,1.0,0.838241,1.095511,0.0,0.000000,0.00000,0.00000,0.838241,0.0
000a9f1e0b14,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1.0,0.0,0.000000,1.266184,0.0,0.000000,0.00000,0.00000,0.000000,0.0


In [10]:
# transform with log1p
df_log = df_div.applymap(np.log1p)
df_log.head(10)

,ENSG00000121410_A1BG,ENSG00000268895_A1BG-AS1,ENSG00000175899_A2M,ENSG00000245105_A2M-AS1,ENSG00000166535_A2ML1,ENSG00000128274_A4GALT,ENSG00000094914_AAAS,ENSG00000081760_AACS,ENSG00000109576_AADAT,ENSG00000103591_AAGAB,...,ENSG00000153975_ZUP1,ENSG00000086827_ZW10,ENSG00000174442_ZWILCH,ENSG00000122952_ZWINT,ENSG00000198205_ZXDA,ENSG00000198455_ZXDB,ENSG00000070476_ZXDC,ENSG00000162378_ZYG11B,ENSG00000159840_ZYX,ENSG00000074755_ZZEF1
000141b198e6,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.693147,0.0,0.000000,...,0.000000,0.000000,0.000000,0.820642,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0003fcc59086,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.611110,...,0.000000,0.693147,0.611110,0.812027,0.0,0.000000,0.000000,0.000000,0.000000,0.0
000441bba05a,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.693147,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0004897a2b2b,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.693147,0.0
000517abfb58,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.693147,0.0,0.000000,...,0.000000,0.000000,0.693147,0.000000,0.0,0.000000,0.000000,0.693147,0.693147,0.0
00056fb385f2,0.0,0.000000,0.0,0.0,0.0,0.0,0.693147,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.693147,0.000000,0.693147,0.0
0005b45cd15b,0.0,0.000000,0.0,0.0,0.0,0.0,0.743870,0.693147,0.0,0.000000,...,0.000000,0.000000,0.693147,0.857814,0.0,0.000000,0.601295,0.601295,0.693147,0.0
00097bbec9a2,0.0,0.000000,0.0,0.0,0.0,0.0,0.693147,0.000000,0.0,0.608943,...,0.000000,0.693147,0.000000,0.608943,0.0,0.608943,0.000000,0.000000,0.693147,0.0
0009a974f266,0.0,0.000000,0.0,0.0,0.0,0.0,0.608809,0.608809,0.0,0.000000,...,0.693147,0.693147,0.608809,0.739797,0.0,0.000000,0.000000,0.000000,0.608809,0.0
000a9f1e0b14,0.0,0.693147,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.693147,0.000000,0.000000,0.818097,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [11]:
df_train_input = df_log

In [12]:
# inner join
# df_merged = df_train_input.merge(df_train_target,how='inner')#,lsuffix='l_',rsuffix='r_')
df_merged = pd.merge(df_train_input,df_train_target,left_index=True, right_index=True)
df_merged.head(100)

,ENSG00000121410_A1BG,ENSG00000268895_A1BG-AS1,ENSG00000175899_A2M,ENSG00000245105_A2M-AS1,ENSG00000166535_A2ML1,ENSG00000128274_A4GALT,ENSG00000094914_AAAS,ENSG00000081760_AACS,ENSG00000109576_AADAT,ENSG00000103591_AAGAB,...,CD94,CD162,CD85j,CD23,CD328,HLA-E,CD82,CD101,CD88,CD224
000141b198e6,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.693147,0.0,0.00000,...,0.144601,10.489143,0.939881,0.841505,3.309048,1.978804,13.027639,-0.208923,2.542908,8.071543
0003fcc59086,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.61111,...,-0.595230,6.023785,0.408263,-0.497212,-0.133765,2.195598,9.705816,-0.580132,7.645536,5.570200
000441bba05a,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,...,0.814176,9.285316,0.299309,1.803804,0.051648,1.212216,3.949074,1.440157,3.350088,0.584580
0004897a2b2b,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,...,4.829338,5.624609,2.015439,4.492601,-1.151961,3.715674,5.097805,1.668119,1.643337,5.632807
000517abfb58,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.693147,0.0,0.00000,...,-1.066876,7.712044,-0.332271,-0.608914,2.938849,0.370199,7.530865,-0.520287,2.078689,2.115581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0054b21867c8,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,...,0.473038,2.826349,0.116303,0.640916,-0.020092,-0.048039,3.671904,1.891259,3.929412,2.359597
005569028ddd,0.0,0.0,0.0,0.693147,0.0,0.0,0.693147,0.000000,0.0,0.00000,...,-0.148417,10.783627,1.109807,0.961539,0.070386,1.774466,4.031822,0.486662,3.392267,5.753120
005633f1ea16,0.0,0.0,0.0,0.000000,0.0,0.0,0.611884,0.000000,0.0,0.00000,...,0.593003,6.540831,2.188413,0.337904,-0.449624,-0.181593,5.820107,-0.139723,5.907623,2.739270
0057cff787fe,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,...,-0.366140,0.343975,0.556323,-1.113267,0.377743,-0.448942,0.539068,0.497742,2.360866,0.351199


In [13]:
#calculate corr coef
target_columns = "','".join(list(axis0_arr))

# corr_df = df_merged.corr()[target_columns]
# corr_threshold = .42
# hcr_df = df_merged.loc[(corr_df.abs() > corr_threshold)]

In [14]:
df_merged.head()
len(df_merged)

3000

In [17]:
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor

target_columns = df_train_target.columns.values
print(target_columns)
# Separate the input and target columns
X = df_merged.drop(target_columns, axis=1)
y = df_merged[target_columns]

# Initialize the MLPRegressor and CatBoostRegressor models
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500)
# catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.1)

# Train the models on the dataset
mlp_model.fit(X, y)
# catboost_model.fit(X, y)

# Predict the target values using the trained models
mlp_preds = mlp_model.predict(X)
# catboost_preds = catboost_model.predict(X)

# mlp1_score = mlp_model.score(test_features, test_target1)
# mlp2_score = mlp2.score(test_features, test_target2)

print(mlp_preds[:100])

['CD86' 'CD274' 'CD270' 'CD155' 'CD112' 'CD47' 'CD48' 'CD40' 'CD154'
 'CD52' 'CD3' 'CD8' 'CD56' 'CD19' 'CD33' 'CD11c' 'HLA-A-B-C' 'CD45RA'
 'CD123' 'CD7' 'CD105' 'CD49f' 'CD194' 'CD4' 'CD44' 'CD14' 'CD16' 'CD25'
 'CD45RO' 'CD279' 'TIGIT' 'Mouse-IgG1' 'Mouse-IgG2a' 'Mouse-IgG2b'
 'Rat-IgG2b' 'CD20' 'CD335' 'CD31' 'Podoplanin' 'CD146' 'IgM' 'CD5'
 'CD195' 'CD32' 'CD196' 'CD185' 'CD103' 'CD69' 'CD62L' 'CD161' 'CD152'
 'CD223' 'KLRG1' 'CD27' 'CD107a' 'CD95' 'CD134' 'HLA-DR' 'CD1c' 'CD11b'
 'CD64' 'CD141' 'CD1d' 'CD314' 'CD35' 'CD57' 'CD272' 'CD278' 'CD58' 'CD39'
 'CX3CR1' 'CD24' 'CD21' 'CD11a' 'CD79b' 'CD244' 'CD169' 'integrinB7'
 'CD268' 'CD42b' 'CD54' 'CD62P' 'CD119' 'TCR' 'Rat-IgG1' 'Rat-IgG2a'
 'CD192' 'CD122' 'FceRIa' 'CD41' 'CD137' 'CD163' 'CD83' 'CD124' 'CD13'
 'CD2' 'CD226' 'CD29' 'CD303' 'CD49b' 'CD81' 'IgD' 'CD18' 'CD28' 'CD38'
 'CD127' 'CD45' 'CD22' 'CD71' 'CD26' 'CD115' 'CD63' 'CD304' 'CD36'
 'CD172a' 'CD72' 'CD158' 'CD93' 'CD49a' 'CD49d' 'CD73' 'CD9' 'TCRVa7.2'
 'TCRVd2' 'LOX-

In [ ]:
#numero uno leaderboard preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

df_norm = StandardScaler().fit_transform(df_train_input.values)
df_median = np.median(df_norm, axis=0)
df_norm_median = df_norm - df_median

tsvd = TruncatedSVD(n_components=128)
df_tsvd = tsvd.fit_transform(df_norm_median)

df

